In [2]:
#default_exp haystack_search

In [3]:
#export
import pprint
import numpy as np
import pandas as pd
import requests
import torch
from sklearn import metrics
from nltk import tokenize
from operator import itemgetter

from haystack.database.elasticsearch import ElasticsearchDocumentStore
from haystack.database.memory import InMemoryDocumentStore

from haystack.retriever.dense import EmbeddingRetriever
from pytorch_hackathon import rss_feeds

import seaborn as sns

100%|██████████| 16/16 [00:09<00:00,  1.60it/s]
/home/piotr/Documents/pytorch_hackathon/pytorch_hackathon/rss_feeds.py:64: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 64 of the file /home/piotr/Documents/pytorch_hackathon/pytorch_hackathon/rss_feeds.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  feed_df['text'] = feed_df['summary'].apply(lambda s: bs4.BeautifulSoup(s).text)


In [4]:
pd.set_option('max_colwidth', 100)

In [5]:
cm = sns.light_palette("green", as_cmap=True)

In [6]:
%cd ..

/home/piotr/Documents/pytorch_hackathon


In [7]:
!ls data

feeds.txt  topics.txt


In [8]:
rss_feed_urls = list(pd.read_table('data/feeds.txt', header=None).iloc[:,0].values)

In [9]:
feed_df = rss_feeds.get_feed_df(rss_feed_urls)

100%|██████████| 16/16 [00:10<00:00,  1.55it/s]


In [10]:
use_gpu = torch.cuda.is_available()

In [11]:
pretty_print = pprint.PrettyPrinter(indent=2).pprint

In [13]:
feed_df.head()

,title,title_detail,links,link,summary,summary_detail,id,guidislink,tags,text,...,comments,authors,author,author_detail,updated,updated_parsed,content,href,media_thumbnail,date
0,PARADE: Passage Representation Aggregation for Document Reranking,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/parade-pass...",https://paperswithcode.com/paper/parade-passage-representation-aggregation-for,"We present PARADE, an end-to-end Transformer-based model that considers document-level context f...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...",https://paperswithcode.com/paper/parade-passage-representation-aggregation-for,False,NaN,"We present PARADE, an end-to-end Transformer-based model that considers document-level context f...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-23
1,MATNet: Motion-Attentive Transition Network for Zero-Shot Video Object Segmentation,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/matnet-moti...",https://paperswithcode.com/paper/matnet-motion-attentive-transition-network,"To further demonstrate the generalization ability of our spatiotemporal learning framework, we e...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...",https://paperswithcode.com/paper/matnet-motion-attentive-transition-network,False,"[{'term': 'Semantic segmentation', 'scheme': None, 'label': None}, {'term': 'Unsupervised video ...","To further demonstrate the generalization ability of our spatiotemporal learning framework, we e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-23
2,Yet Another Intermediate-Level Attack,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/yet-another...",https://paperswithcode.com/paper/yet-another-intermediate-level-attack,The transferability of adversarial examples across deep neural network (DNN) models is the crux ...,"{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...",https://paperswithcode.com/paper/yet-another-intermediate-level-attack,False,NaN,The transferability of adversarial examples across deep neural network (DNN) models is the crux ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-23
3,Uncertainty Estimation in Medical Image Denoising with Bayesian Deep Image Prior,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/uncertainty...",https://paperswithcode.com/paper/uncertainty-estimation-in-medical-image,We use a randomly initialized convolutional network as parameterization of the reconstructed ima...,"{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...",https://paperswithcode.com/paper/uncertainty-estimation-in-medical-image,False,"[{'term': 'Image denoising', 'scheme': None, 'label': None}]",We use a randomly initialized convolutional network as parameterization of the reconstructed ima...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-23
4,Simultaneous Detection and Tracking with Motion Modelling for Multiple Object Tracking,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/simultaneou...",https://paperswithcode.com/paper/simultaneous-detection-and-tracking-with,Deep learning-based Multiple Object Tracking (MOT) currently relies on off-the-shelf detectors f...,"{'type': 'text/html', 'language': None, 'base':

In [14]:
#export



class Searcher:
    
    def __init__(
        self,
        model_name,
        text_col,
        use_gpu,
        max_document_length=256,
        quantize_model=True,
        document_store_cls=InMemoryDocumentStore
    ):
        self.text_col = text_col
        self.embedding_col = text_col + '_emb'
        self.max_document_length = max_document_length
        self.model_name = model_name
        self.document_store = document_store_cls(
            embedding_field=self.embedding_col,
        )
        self.retriever = self._setup_retriever(use_gpu, quantize_model)

    def _setup_retriever(self, use_gpu, quantize_model):
        retriever = EmbeddingRetriever(
            document_store=self.document_store,
            embedding_model=self.model_name,
            use_gpu=use_gpu)
        if not use_gpu and quantize_model:
            self.set_quantized_model(retriever)
            
        return retriever

    def add_texts(
        self,
        df
    ):
        truncated_texts = [
            ' '.join(tokenize.wordpunct_tokenize(text)[:self.max_document_length])
            for text in df[self.text_col] 
        ]
        article_embeddings = self.retriever.embed_queries(
            texts=truncated_texts
        )

        df[self.embedding_col] = article_embeddings
        self.document_store.write_documents(df.to_dict(orient='records'))
    
    @classmethod
    def set_quantized_model(cls, retriever):
        quantized_model = torch.quantization.quantize_dynamic(
            retriever.embedding_model.model,
            {torch.nn.Linear}, dtype=torch.qint8
        )
        retriever.embedding_model.model = quantized_model
        
    @classmethod 
    def sigmoid(cls, x):
        return 1 / (1 + np.exp(-x))
    
    @classmethod
    def doc_to_dict(cls, doc):
        d = {}
        d['text'] = doc.text
        d['title'] = doc.meta['title']
        d['score'] = doc.query_score
        d['link'] = doc.meta['link']
        d['date'] = doc.meta['date']
        return d

    def get_topic_score_df(self, raw_results, topic_strings):
        topic_query_strings = [
            'text is about {}'.format(topic)
            for topic in topic_strings
        ]

        results = [
            self.doc_to_dict(doc)
            for doc in raw_results 
        ]
        result_embeddings = np.array([
            doc.meta['text_emb']
            for doc in raw_results
        ]).astype('float32')
        topic_query_embeddings = np.array(self.retriever.embed_passages(
            list(topic_strings)
        )).astype('float32')

        scores_df = pd.DataFrame({})
        scores_df['title'] = list(map(itemgetter('title'), results))
        scores_df['text'] = list(map(itemgetter('text'), results))
        scores_df['link'] = list(map(itemgetter('link'), results))
        scores_df['date'] = list(map(itemgetter('date'), results))

        scores = pd.DataFrame(metrics.pairwise.cosine_similarity(
            result_embeddings,
            topic_query_embeddings
        ))
        scores.columns = topic_strings

        scores_df = pd.concat(
            [scores_df, self.sigmoid(scores)],
            axis=1
        )
        return scores_df

In [13]:
model_name = "deepset/sentence_bert"

In [14]:
searcher = Searcher(
    model_name,
    'text',
    use_gpu=use_gpu
)

08/17/2020 17:55:34 - INFO - haystack.retriever.dense -   Init retriever using embeddings of model deepset/sentence_bert
08/17/2020 17:55:34 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
08/17/2020 17:55:34 - INFO - farm.infer -   Could not find `deepset/sentence_bert` locally. Try to download from model hub ...
08/17/2020 17:55:34 - INFO - transformers.modeling_utils -   loading weights file https://cdn.huggingface.co/deepset/sentence_bert/pytorch_model.bin from cache at /home/kuba/.cache/torch/transformers/fa9d12cb00cd5a31f5a5367f58d242199473a6deb02c51380681ade7bf33c713.4948a08b5d844db1ecda79f6e7f47643f0175f2c030d48ce8b3beee3c6bd6012
08/17/2020 17:55:36 - INFO - transformers.modeling_utils -   All model checkpoint weights were used when initializing BertModel.

08/17/2020 17:55:36 - INFO - transformers.modeling_utils -   All the weights of BertModel were initialized from the model checkpoint at deepset/sentence_be

In [15]:
searcher.add_texts(feed_df)

08/17/2020 17:55:42 - INFO - farm.data_handler.processor -   *** Show 2 random examples ***
08/17/2020 17:55:42 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"       \___ \ / _` | '_ ` _ \| '_ \| |/ _ \ 
       || /\     ____) | (_| | | | | | | |_) | |  __/
    /\ ||//\)   |_____/ \__,_|_| |_| |_| .__/|_|\___|
   (/\||/                             |_|           
______\||/___________________________________________                     

ID: 57-0
Clear Text: 
 	text: Comments
Tokenized: 
 	tokens: ['comments']
 	offsets: [0]
 	start_of_word: [True]
Features: 
 	input_ids: [101, 7928, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Inferencing Samples: 100%|██████████| 74/74 [00:10<00:00,  7.10 Batches/s]


In [16]:
article_texts = feed_df['text']

In [17]:
topic_strings = pd.read_table('data/topics.txt', header=None).iloc[:,0].values

In [18]:
print('\n'.join(topic_strings))

deep learning
natural language processing
computer vision
statistics
implementation
visualization
industry
software engineering
reddit question
arxiv
cloud computing
deployment
competitions
business
business intelligence


In [19]:
topic_query_strings = [
    'text is about {}'.format(topic)
    for topic in topic_strings
]

In [20]:
raw_results = searcher.retriever.retrieve(
    topic_query_strings[1]
)

08/17/2020 17:55:53 - INFO - farm.data_handler.processor -   *** Show 2 random examples ***
08/17/2020 17:55:53 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"       \___ \ / _` | '_ ` _ \| '_ \| |/ _ \ 
       || /\     ____) | (_| | | | | | | |_) | |  __/
    /\ ||//\)   |_____/ \__,_|_| |_| |_| .__/|_|\___|
   (/\||/                             |_|           
______\||/___________________________________________                     

ID: 0-0
Clear Text: 
 	text: text is about natural language processing
Tokenized: 
 	tokens: ['text', 'is', 'about', 'natural', 'language', 'processing']
 	offsets: [0, 5, 8, 14, 22, 31]
 	start_of_word: [True, True, True, True, True, True]
Features: 
 	input_ids: [101, 3793, 2003, 2055, 3019, 2653, 6364, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 33.47 Batches/s]


In [21]:
scores_df = searcher.get_topic_score_df( raw_results, topic_strings)

08/17/2020 17:55:53 - INFO - farm.data_handler.processor -   *** Show 2 random examples ***
08/17/2020 17:55:53 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"       \___ \ / _` | '_ ` _ \| '_ \| |/ _ \ 
       || /\     ____) | (_| | | | | | | |_) | |  __/
    /\ ||//\)   |_____/ \__,_|_| |_| |_| .__/|_|\___|
   (/\||/                             |_|           
______\||/___________________________________________                     

ID: 9-0
Clear Text: 
 	text: arxiv
Tokenized: 
 	tokens: ['ar', '##xi', '##v']
 	offsets: [0, 2, 4]
 	start_of_word: [True, False, False]
Features: 
 	input_ids: [101, 12098, 9048, 2615, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Inferencing Samples: 100%|██████████| 4/4 [00:00<00:00,  7.71 Batches/s]
08/17/2020 17:55:53 - INFO - numexpr.utils -   Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
08/17/2020 17:55:53 - INFO - numexpr.utils -   NumExpr defaulting to 8 threads.


In [22]:
scores_df.style.background_gradient(cmap=cm)

In [23]:
[doc.text for doc in raw_results]

['Natural Language ProcessingAn Introduction to Retrieval-Augmented Language Model Pre-TrainingPhoto by Edward Ma on\xa0UnsplashSince 2018, the transformer-based language model has been proven to achieve good performance in lots of NLP downstream tasks such as Open-domain Question Answer (Open-QA). To achieve better results, models intend to increase model parameters (e.g. more heads, larger dimensions) in order to stored world knowledge in the neural\xa0network.Guu et al. (2020) from Google Research released the state-of-the-art model (Retrieval-Augmented Language Model Pre-Training, aks REALM) which leverages knowledge retriever augmented data from other large corpora such as Wikipedia. Given an extra signal, it helped the model to deliver a better result. In this storied, we will go through how does this model achieves the start-of-the-art result.REALM OverviewThe overall idea is leveraging extra document to provide more signal to the model such that it can predict masked token accu

In [24]:
scores_df

,title,text,link,deep learning,natural language processing,computer vision,statistics,implementation,visualization,industry,software engineering,reddit question,arxiv,cloud computing,deployment,competitions,business,business intelligence
0,REALM: Retrieval-Augmented Language Model Pre-Training,Natural Language ProcessingAn Introduction to Retrieval-Augmented Language Model Pre-TrainingPho...,https://medium.com/towards-artificial-intelligence/realm-retrieval-augmented-language-model-pre-...,0.594145,0.613702,0.534048,0.502438,0.564319,0.550380,0.517819,0.585611,0.549369,0.490301,0.518406,0.505397,0.519020,0.496123,0.518821
1,Evaluating the Impact of Knowledge Graph Context on Entity Disambiguation Models,Pretrained Transformer models have emerged as state-of-the-art approaches that learn contextual ...,https://paperswithcode.com/paper/evaluating-the-impact-of-knowledge-graph,0.613387,0.597722,0.531973,0.501481,0.593792,0.559076,0.545501,0.592912,0.543777,0.490492,0.511187,0.537554,0.530916,0.494104,0.496908
2,"[P] Malay and Indonesian languages Toolkit, malaya.readthedocs.io","Malaya is a Natural-Language-Toolkit library for Malay and Indonesian languages, powered by Deep...",https://www.reddit.com/r/MachineLearning/comments/iav8g5/p_malay_and_indonesian_languages_toolkit/,0.573247,0.606518,0.518175,0.492254,0.534429,0.534387,0.504195,0.553177,0.534189,0.505804,0.495088,0.497745,0.496114,0.497304,0.506360
3,A single legal text representation at Doctrine: the legal camemBERT,"As a legal platform, Doctrine aggregates a lot of legal data with the intent of making them acce...",https://medium.com/doctrine/a-single-legal-text-representation-at-doctrine-the-legal-camembert-a...,0.570231,0.586314,0.521751,0.499045,0.548911,0.511261,0.517473,0.595577,0.505991,0.492187,0.509414,0.505175,0.512034,0.511341,0.525712
4,A Large-Scale Chinese Short-Text Conversation Dataset,The cleaned dataset and the pre-training models will facilitate the research of short-text conve...,https://paperswithcode.com/paper/a-large-scale-chinese-short-text-conversation,0.621184,0.586691,0.545824,0.541583,0.584535,0.550647,0.527896,0.600209,0.548811,0.484411,0.517256,0.519267,0.528229,0.519199,0.530539
5,[D] Why does models like GPT-3 or BERT don't have overfitting problems?,"Hey everyone, I am new to Natural Language Processing, but I have experience in Machine Learning...",https://www.reddit.com/r/MachineLearning/comments/ib4rth/d_why_does_models_like_gpt3_or_bert_don...,0.574753,0.577933,0.525968,0.498621,0.520720,0.489520,0.512251,0.579364,0.521330,0.504109,0.526190,0.485646,0.517864,0.528094,0.550471
6,"[Q] Data scientist here, working on gathering a corpus of academic papers focusing on ""Cognitive...",Hello. I want to collect as many as papers as I can that will fall into this category. The main ...,https://www.reddit.com/r/cognitivelinguistics/comments/ensu0n/q_data_scientist_here_working_on_g...,0.555300,0.578011,0.498356,0.489413,0.526189,0.513210,0.489906,0.544217,0.557324,0.480271,0.496461,0.485666,0.530694,0.496379,0.504614
7,KR-BERT: A Small-Scale Korean-Specific Language Model,"Since the appearance of BERT, recent works including XLNet and RoBERTa utilize sentence embeddin...",https://paperswithcode.com/paper/kr-bert-a-small-scale-korean-specific,0.574174,0.594593,0.522638,0.532562,0.583562,0.552562,0.547111,0.580898,0.551625,0.498453,0.515522,0.564181,0.550184,0.508991,0.513304
8,Dialogue State Induction Using Neural Latent Variable Models,Dialogue state modules are a useful component in a task-oriented dialogue system. Code: https://...,https://paperswithcode.com/paper/dialogue-state-induction-using-neural-latent,0.609547,0.595336,0.546996,0.492073,0.601822,0.568787,0.546473,0.602195,0.570310,0.499499,0.532563,0.543948,0.563158,0.529331,0.538693
9,[D] Supervised Approach to Extractive Text Summarisation | Research Paper Walkthrough,Automatic text summarisation is the task of shortening the given piece of text yet retaining the...,https://www.redd